<a href="https://www.kaggle.com/code/hari141v/open-sourced-llms-mistral-7b-instruct-v0-1?scriptVersionId=187244757" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Import necessary libraries and packages



In [1]:
pip install -U transformers accelerate bitsandbytes mistral-common accelerate mistral_inference  -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cudf 24.4.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which is incompatible.
jupyterlab 4.2.1 req

#### Get file from LLM zoomcamp course repo

In [2]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-07 16:42:08--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-07 16:42:08 (48.9 MB/s) - 'minsearch.py' saved [3832/3832]



#### Get json file from course repo and create TF-IDF

In [3]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

#### search query based on cosine-similarity

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, BitsAndBytesConfig,  pipeline
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-07-07 16:42:16.917867: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 16:42:16.917976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 16:42:17.033493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

#### Mistral models can be downloaded with huggingface authenticatiion

In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

In [7]:
from huggingface_hub import login
login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Get Mistral-7B-Instruct-v0.1 from huggingface

In [8]:
 
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             quantization_config=quantization_config,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### Create pipeline

In [9]:
generator = pipeline("text-generation", model=model, 
                     tokenizer=tokenizer
                    )

#### Build prompt

In [21]:
def build_prompt(query, search_results):
    prompt_template = [
 {"role": "system", "content": "Your response should be clear and concise"},
 

    ]

   
    
    for doc in search_results:
        prompt_template.append({'role':'user','content':doc['question']})
        prompt_template.append({'role':'assistant','content':doc['text']})
        
    prompt_template.append({'role':'user','content':query})
    prompt = tokenizer.apply_chat_template(prompt_template, tokenize=False,
            add_generation_prompt=True)
    return prompt

def llm(prompt,temp):
    config = {'max_length':1000,
                'temperature':temp,
                'do_sample':True,
                'num_return_sequences':1,
                'num_beams':1,         
                'top_p':0.95,         
                'top_k':50,        
                'eos_token_id':tokenizer.eos_token_id,
                'pad_token_id':tokenizer.eos_token_id,
               }
    response = generator(prompt,**config)
    response_final = response[0]['generated_text'].split('[/INST]')[-1].strip()
    return response_final

#### Get results and create prompt from it and generate result using llm

In [22]:
def rag(query,temp=0.1):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt,temp)
    return answer

#### The following result based on the json document data

#### Temperature 0.1

In [23]:
%%time
print(rag("I just discovered the course. Can I still enroll it?"))

Yes, you can still enroll in the course even if it has already started.
You can still submit the homeworks and complete the course.
Please note that there will be deadlines for turning in the final projects, so it's better to start working on them as soon as possible.
CPU times: user 6.56 s, sys: 456 ms, total: 7.01 s
Wall time: 7.01 s


#### Let's try using different temperature values and examine the differences in the results.

In [13]:
import numpy as np
from IPython.display import display, Markdown

In [24]:
%%time
for temp in np.round(np.linspace(0.2,1.0,9),1):
    display(Markdown(f"### Temperature: {temp}"))
    result=rag('I just discovered the course. Can I still enroll it?',temp)
    display(Markdown(f"Result: {result}"))
    print("-"*100)

### Temperature: 0.2

Result: Yes, you can still enroll in the course even if it has already started.
You can check the course schedule and see if there are any upcoming office hours or live sessions that you can attend.
You can also check the course materials and start working on the assignments at your own pace.
It's important to note that some assignments may have deadlines, so you may want to start working on them as soon as possible to ensure that you can submit them on time.

----------------------------------------------------------------------------------------------------


### Temperature: 0.3

Result: Yes, you can still enroll in the course.
The course will start on 15th Jan 2024 at 17h00.
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.
You can also check the course schedule and the course content.
Please note that the course is designed for intermediate to advanced learners.

...

Result: Yes, you can still enroll even if the course has already started.
You will be able to submit homeworks and participate in class activities. However, be aware that there will be deadlines for turning in the final projects and assignments.
You can also still look over the homeworks and continue preparing for the next cohort.
I guess you can also start working on your final capstone project.

----------------------------------------------------------------------------------------------------


### Temperature: 1.0

Result: Yes, you can enroll the course any time after it has started.
The course will run for a continuous period.
You can enroll at any time you'd like, as long as you meet the prerequisites for the course.
You can start on the lessons any time you'd like and work through the homework at your own pace.

----------------------------------------------------------------------------------------------------
CPU times: user 1min 10s, sys: 3.96 s, total: 1min 14s
Wall time: 1min 14s
